# CNN


CNN 通常用于计算机视觉任务。CNN 通过在输入数据上滑动卷积核来提取特征。卷积核是一个小的矩阵，它在输入数据上滑动，计算输入数据和卷积核之间的点积。卷积核的大小通常是 3x3 或 5x5。卷积核的大小和数量是超参数，需要根据具体任务进行调整。CNN 的变体有： ResNets、DenseNets 和 Octave 卷积。

## Neural Networks and Representation Learning

在之前的章节中，展示了两个例子，第一个是房屋预测，第二个是 MNIST 预测。在这两个例子中，带隐藏层的神经网络效果均要好于不带隐藏层的神经网络。原因是隐藏层提取了输入数据和输出数据之间的非线性的特征。

然而，一个更普遍的原因是，在机器学习中，我们经常需要原始特征的线性组合，以便有效地预测我们的目标。例如在 MNIST 中，单张图片有 784 个像素，不同像素的组合可以表示不同的数字，所有这些组合都会对图像是某一特定数字的概率产生积极或消极的影响。神经网络可以通过训练过程自动发现重要的原始特征组合。这一过程首先是通过与随机权重矩阵相乘来创建原始特征的初始随机组合；通过训练，神经网络学会提炼出有用的组合，并摒弃那些没用的组合。这种学习哪些特征组合是重要的过程被称为 `表征学习`，也是神经网络在不同领域取得成功的主要原因。

![CNN](./images/05_cnn1.png)


### A Different Architecture for Image Data


同样也是创建特征的组合，但是是在更高层次并且是更高的数量级上。每个特征的组合只是输入图像中一小块矩形区域的像素组合。

![CNN](./images/05_cnn2.png)

让神经网络学习所有特征的组合，也就是所有像素的组合是困难且低效的，且忽略了前面的一个见解（Insight）：图像中大部分重要的特征组合都出现在小矩形图像中。从这些小矩形中抽取特征并组合的操作是卷积操作。卷积操作是一种特殊的矩阵乘法，它在输入数据上滑动一个小的矩阵（卷积核），计算输入数据和卷积核之间的点积。卷积核的大小通常是 3x3 或 5x5。卷积核的大小和数量是超参数，需要根据具体任务进行调整。


### The Convolution Operation


在设定卷积核大小后，卷积核应用在图像上的不同位置。例如我们有以下输入图像的数据：
$$
I = \begin{bmatrix}
i_{1,1} & i_{1,2} & i_{1,3} & i_{1,4} & i_{1,5} \\
i_{2,1} & i_{2,2} & i_{2,3} & i_{2,4} & i_{2,5} \\
i_{3,1} & i_{3,2} & i_{3,3} & i_{3,4} & i_{3,5} \\
i_{4,1} & i_{4,2} & i_{4,3} & i_{4,4} & i_{4,5} \\
i_{5,1} & i_{5,2} & i_{5,3} & i_{5,4} & i_{5,5} \\
\end{bmatrix}
$$

假设我们有一个 3x3 的卷积核：
$$
K = \begin{bmatrix}
k_{1,1} & k_{1,2} & k_{1,3} \\
k_{2,1} & k_{2,2} & k_{2,3} \\
k_{3,1} & k_{3,2} & k_{3,3} \\
\end{bmatrix}
$$

假设我们将卷积核作用在 $I$ 中间的 (3,3) 位置，结果记为 $o_33$ （o 表示 output）
$$
o_{33} = w_{1,1} \times i_{2,2} + w_{1,2} \times i_{2,3} + w_{1,3} \times i_{2,4} + w_{2,1} \times i_{3,2} + w_{2,2} \times i_{3,3} + w_{2,3} \times i_{3,4} + w_{3,1} \times i_{4,2} + w_{3,2} \times i_{4,3} + w_{3,3} \times i_{4,4}
$$

这个值会像我们在神经网络中看到的其他计算特征一样被处理：它可能会被添加一个偏置，然后可能会被送入一个激活函数，然后它将代表一个 "神经元 "或 "学习特征"，并传递给网络的后续层

根据不同卷积核的类型，也称为"模式提取"（pattern detectors），可以提取图像中不同属性的特征。例如下面的 3x3 卷积核可以检测图像中的边缘：
$$
K = \begin{bmatrix}
0& 1 & 0 \\
1 & 4 & 1 \\
0 & 1 & 0 \\
\end{bmatrix}
$$

已知还有类似的矩阵可以检测是否存在角、是否存在垂直线或水平线，等等。这些卷积核的选择是超参数，需要根据具体任务进行调整。

现在假设我们使用同一组权重W来检测由W定义的视觉模式是否存在于输入图像中的每个位置。我们可以想象“在输入图像上滑动 W”，取 W 与图像每个位置处的像素的点积，最终得到一个与原始图像大小几乎相同的新图像 O（可能略有不同，具体取决于关于我们如何处理边缘）。图像 O 将是一种 "特征图"，显示输入图像中出现 W 所定义图案的位置。这种操作实际上就是卷积神经网络中的操作；它被称为卷积，其输出实际上被称为特征图。

在进入 `Operation` 前，我们给卷积再增加点维度。

## Multichannel Convolution Operation

回顾一下：卷积神经网络与普通神经网络的不同之处在于，它们创建的特征要多出一个数量级，而且每个特征都是输入图像中一小块区域的函数。现在我们可以说得更具体一些：从 n 个输入像素开始，刚才描述的卷积操作将创建 n 个输出特征，输入图像中的每个位置（区域）都有一个特征。

在神经网络的卷积层中实际发生的情况则更进一步：在这里，我们将创建 n 个特征的 f 个集合，每个集合都有相应的（初始随机的）权重集，这些权重集定义了一种视觉模式，输入图像中每个位置的检测结果都将被捕捉到特征图中。这 f 个特征图将通过 f 次卷积操作创建。这些特征图将被堆叠在一起，形成一个新的多通道特征图，这个特征图将被传递到网络的下一层。

![CNN](./images/05_cnn3.png)

通过特定权重集检测到的每个 "特征集 "被称为一个特征图，而在卷积层中，特征图的数量被称为卷积层的通道数--这就是为什么卷积层所涉及的操作被称为多通道卷积。此外，f 组权重 W 被称为卷积滤波器（convolutional filters）。

## Convolutional Layers

在我们之前的模型中，`Layer` 接收二维的 `ndarray` 作为输入，输出一个二维的 `ndarray`。在卷积神经网络中，`Layer` 接收三维的 `ndarray` 作为输入，输出一个三维的 `ndarray`。这个三维的 `ndarray` 有两个维度是空间维度（高、宽），一个维度是通道维度。这里就引发了另外一个问题：我们要如何将这个 `ndarray` 传递给下一层的卷积网络从而创建 “深度卷积” 神经网络？

如果卷积层的输出是一个 m 个通道 × 图像高度 × 图像宽度的三维数组，那么图像中 m 个特征图中的一个给定位置就是对上一层中每个相应特征图中的该位置进行不同滤波器卷积的线性组合。这样，m 个滤波器图中的每个位置就代表了先前卷积层已学习到的 m 个视觉特征的组合。

### 多通道卷积的操作

#### 输入参数的维度
- Batch Size
- Input channels
- Image height
- Image width

#### 输出参数的维度
- Batch Size
- Output channels
- Image height
- Image width

### 卷积核的维度
- Input channels
- Output channels
- Kernel height
- Kernel width

### 卷积层与全连接层的区别

![CNN](./images/05_cnn4.png)


此外，这两种神经层的最后一个区别在于对单个神经元本身的解释方式：

- 对全连接层中神经元的解释是，它检测当前观测结果中是否存在前一层所学特征的特定组合。
- 对卷积层中神经元的解释是，它检测输入图像的给定位置是否存在前一层学习到的特定视觉模式组合。


将卷基层纳入神经网络之前，还需要解决一个问题：如何利用我们获得的输出维度的 `ndarray` 进行预测？

### 使用卷积层进行预测：Flatten Layer

在上一章中使用全连接神经网络预测图像属于 10 个类别中的哪一个时，我们只需确保最后一层的维度为 10；然后我们就可以将这 10 个数字输入 softmax cross entropy 函数，以确保它们被解释为概率。

在卷积层中，我们有一个三维 `ndarray`，每个观测点的形状为 m 个通道 × 图像高度 × 图像宽度，每个神经元只是表示在图像的特定位置是否存在视觉特征的特定组合（如果是深度卷积神经网络，则可能是特征的特征或特征的特征）。这与我们将一个全连接神经网络应用于该图像所学习到的特征并无不同：第一个全连接层代表单个像素的特征，第二个全连接层代表这些特征的特征，以此类推。而在全连接架构中，我们只需将网络学习到的每个 "特征的特征" 视为一个神经元，将其作为预测图像所属类别的输入。

在卷积神经网络中也是一样，我们将 `m` 个特征图视为 m 个通道 × 图像高度 × 图像宽度个神经元，使用 `Flatten` 操作将其转换为一个一维 `ndarray`，然后将其输入到 softmax cross entropy 函数中，以确保它们被解释为概率。

在进入实现 `Flatten` 操作前，我们再看个在卷积神经网络中常用的操作：`Pooling`。

### Pooling Layers

池化层是卷积神经网络中常用的另一种层。它们对卷积操作创建的每个特征图进行简单的降采样；最常用的池化大小为 2，这涉及将每个特征图的每个 2 × 2 部分映射为该部分的最大值（在最大池化的情况下）或该部分的平均值（在平均池化的情况下）。

![Pooling](./images/05_pooling.png)


池化层的主要有点在于降低了计算量。但也有很多人对池化层的必要性提出了置疑。现如今很多的 CNN 已经很少或者没有了池化层。


### 将 CNN 应用于图像之外

到目前为止，我们所描述的一切，都是使用神经网络处理图像的极其标准的方法：图像通常表示为一组 m 个像素通道，其中 m = 1 表示黑白图像，m = 3 表示彩色图像，然后对每个通道进行一定数量的 m 次卷积运算（使用前面解释过的 m1×m2 滤波图），这种模式会持续几个层。这些知识点在其他的资料中也会介绍，但是它们没介绍的是，CNN 也会被利用在图像之外的任务中，比如 DeepMind 的 AlphaGo 就是使用 CNN 来处理围棋的棋盘状态。

CNN 及其多通道卷积操作通常应用于图像，但用多个 "通道 "来表示沿某个空间维度排列的数据这一更为普遍的想法甚至适用于图像之外的其他领域。

### 实现多通道卷积操作

- 执行卷积操作
- 填充

In [1]:
import numpy as np
from numpy import ndarray

In [2]:
def assert_same_shape(output: ndarray,
                      output_grad: ndarray):
    assert output.shape == output_grad.shape, \
        '''
        Two ndarray should have the same shape; instead, first ndarray's shape is {0}
        and second ndarray's shape is {1}.
        '''.format(tuple(output_grad.shape), tuple(output.shape))
    return None


def assert_dim(t: ndarray,
               dim: ndarray):
    assert len(t.shape) == dim, \
        '''
        Tensor expected to have dimension {0}, instead has dimension {1}
        '''.format(dim, len(t.shape))
    return None

#### 1D Convolution

##### Padding

In [3]:
# 1 input， 1 output

input_1d = np.array([1, 2, 3, 4, 5])
kernel_1d = np.array([1, 1, 1])

In [6]:
def _pad_1d(input: ndarray,
            padding: int) -> ndarray:
    zero = np.array([0])
    zero = np.repeat(zero, padding)
    return np.concatenate([zero, input, zero])

In [5]:
_pad_1d(input_1d, 1)

array([0, 1, 2, 3, 4, 5, 0])

##### Forward

In [12]:
def conv_1d(input: ndarray, param: ndarray) -> ndarray:
    # assert correct dimensions
    assert_dim(input, 1)
    assert_dim(param, 1)

    # pad the input
    param_len = param.shape[0]
    param_mid = param_len // 2
    input_padded = _pad_1d(input, param_mid)

    # initialize the output
    output = np.zeros(input.shape)

    # perform the 1d convolution
    for o in range(output.shape[0]):
        for p in range(param_len):
            output[o] += param[p] * input_padded[o + p]

    assert_same_shape(output, input)
    return output

In [8]:
def conv_1d_sum(input: ndarray, param: ndarray) -> ndarray:
    return np.sum(conv_1d(input, param))

In [13]:
conv_1d_sum(input_1d, kernel_1d)

39.0

#### Stride

在介绍反向传播前，我们先来看看卷积操作的另一个超参数：Stride。Stride 是卷积核在输入数据上滑动的步长。在之前的例子中，Stride 是 1，也就是卷积核每次移动一个位置。如果 Stride 是 2，那么卷积核每次移动两个位置。Stride 是一个超参数，需要根据具体任务进行调整。

步长的大小也会影响输出的大小。假设输入数据的大小是 $n$，卷积核的大小是 $f$，填充的大小是 $p$，步长是 $s$，那么输出数据的大小是：
$$
\frac{n - f + 2p}{s} + 1
$$

##### 卷积：向后传播

我们需要计算
- 损失相对于卷积运算输入的每个元素的偏导数
- 损失相对于卷积核的偏导数

In [15]:
np.random.seed(190220)
print(np.random.randint(0, input_1d.shape[0]))
print(np.random.randint(0, kernel_1d.shape[0]))

4
0


In [16]:
input_1d_2 = np.array([1, 2, 3, 4, 6])
param_1d = np.array([1, 1, 1])

In [17]:
print(conv_1d_sum(input_1d_2, param_1d) - conv_1d_sum(input_1d_2, param_1d))

0.0


In [18]:
input_1d = np.array([1, 2, 3, 4, 5])
param_1d_2 = np.array([2, 1, 1])

print(conv_1d_sum(input_1d, param_1d_2) - conv_1d_sum(input_1d, param_1d))

10.0


## Training Convolutional Neural Networks